In [3]:
! pip install deap
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.1 MB/s eta 0:00:00


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import scipy.stats as stats
import statsmodels.stats.multicomp as mc
# import pairwise_tukeyhsd


from deap import base
from deap import creator
from deap import tools

In [5]:
# Problema de maximização do fitness
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
# Indivíduo derivado de uma lista
creator.create("Individual", list, fitness=creator.FitnessMax)

In [6]:
toolbox = base.Toolbox()
# Attribute generator
# gerador gera um atributo inteiro binário
toolbox.register("attr_bool", random.uniform, -30, 30)
# Structure initializers
# gerador de indivíduos considera 100 posições binárias na lista
toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.attr_bool, 30)
# população é uma lista de indivíduos
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [7]:
def ackley_function(individual):
    a = 20
    b = 0.2
    c = 2*np.pi

    first_term = -a * np.exp(-b * np.sqrt(np.mean(np.square(individual))))
    second_term = -np.exp(np.mean(np.cos(c * np.array(individual))))

    return first_term + second_term + a + np.exp(1),


In [8]:
def mutGau(individual,sigma,indpb):
  for i in range(len(individual)):
    if random.random() < indpb:
      individual[i] = random.gauss(individual[i],sigma*individual[i])
  return individual,

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutGau, sigma=0.3, indpb=0.4)
toolbox.register("select", tools.selTournament, tournsize=30)
toolbox.register("evaluate", ackley_function)


In [9]:
def funcion(population,generations):
    # fixa a semente
    # random.seed(64)

    # gera população de 300 indivíduos
    pop = toolbox.population(n=population)

    # cria um log para armazenar dados estatísticos
    log = tools.Logbook()

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2

    # Extracting all the fitnesses of
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0

    # Begin the evolution
    while min(fits) > 0 and g < generations:
        # A new generation
        g = g + 1
        # print("-- Generation %i --" % g)

        # Número de indivíduos de elite para preservar
        num_elites = 5  # Você pode ajustar este valor

        # Selecionar os indivíduos de elite
        # elites = tools.selBest(pop, num_elites)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))


        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring  # Elites já estão em offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        # print("  Min %s" % min(fits))
        # print("  Max %s" % max(fits))
        # print("  Avg %s" % mean)
        # print("  Std %s" % std)

        # Record statistics
        pop_fitness = pop.copy()
        pop_fitness.sort(key=lambda x: x.fitness.getValues()[0])

        log.record(agen=g, ind=pop_fitness[0], fit=pop_fitness[0].fitness.getValues()[0])

    gen = log.select("agen")
    fit = log.select("fit")

    print(f"Terminado um experimento com tamPop={population} e nGer={generations}")

    return gen,fits

In [10]:
# Parâmetros
tamPops = [50, 100, 150]
nGers = [1000, 2000, 3000]
nTestes = 30

results = []

for i, tamPop in enumerate(tamPops):
    for j, nGer in enumerate(nGers):
      unit_result = []
      for _ in range(nTestes):
          _, fitVal = funcion(tamPop, nGer)
          fitVal = min(fitVal)
          unit_result.append(fitVal)
      results.append(unit_result)

Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com tamPop=50 e nGer=1000
Terminado um experimento com ta

In [11]:
results

[[3.2013331375475933,
  3.280890773815653,
  3.9968028886505635e-15,
  3.9968028886505635e-15,
  3.9968028886505635e-15,
  2.8237088744681462,
  2.1737893707296507e-08,
  3.9968028886505635e-15,
  3.9968028886505635e-15,
  2.587335061685057,
  3.9968028886505635e-15,
  3.9968028886505635e-15,
  3.4042635578531244,
  2.5786042065134898,
  2.9337506958963115,
  3.9968028886505635e-15,
  4.440892098500626e-16,
  4.440892098500626e-16,
  7.549516567451064e-15,
  9.82258718806861e-11,
  3.241851231905457e-14,
  3.9968028886505635e-15,
  1.4346892562855373e-09,
  2.8664002960200823,
  2.9007554291014306,
  2.670379535740065,
  3.9968028886505635e-15,
  2.937745571758239,
  7.549516567451064e-15,
  3.1290691968663435],
 [3.349509775931779,
  3.517469031149471,
  7.549516567451064e-15,
  4.440892098500626e-16,
  2.9453150285878285,
  4.440892098500626e-16,
  4.440892098500626e-16,
  3.4026380207180478,
  3.2322512047628185,
  4.440892098500626e-16,
  4.440892098500626e-16,
  3.9968028886505635

In [12]:
# @title Testes estatisticos (ANOVA)


from scipy.stats import f_oneway
grau_confianca = 0.05

f, p_valor = f_oneway(*results)

print(f"Valor F: {f}; p-valor: {p_valor}")

if p_valor < grau_confianca:
  print("Há diferença considerável! Conduza o teste de tukey")
else:
  print("Não há diferença considerável. Não há necessidade de conduzir o teste de tukey")

Valor F: 3.0823634735607586; p-valor: 0.0024466088905243637
Há diferença considerável! Conduza o teste de tukey


In [13]:
# @title Testes estatisticos (Tukey)
from scipy.stats import tukey_hsd

tukey_results = tukey_hsd(*results)

print(tukey_results)

Tukey's HSD Pairwise Group Comparisons (95.0% Confidence Interval)
Comparison  Statistic  p-value  Lower CI  Upper CI
 (0 - 1)      0.324     0.990    -0.747     1.395
 (0 - 2)      0.850     0.245    -0.220     1.921
 (0 - 3)     -0.050     1.000    -1.121     1.021
 (0 - 4)      0.665     0.585    -0.406     1.736
 (0 - 5)      0.893     0.188    -0.178     1.964
 (0 - 6)     -0.312     0.992    -1.383     0.759
 (0 - 7)      0.447     0.929    -0.624     1.518
 (0 - 8)      0.631     0.653    -0.440     1.702
 (1 - 0)     -0.324     0.990    -1.395     0.747
 (1 - 2)      0.526     0.837    -0.545     1.597
 (1 - 3)     -0.375     0.975    -1.446     0.696
 (1 - 4)      0.341     0.986    -0.730     1.412
 (1 - 5)      0.569     0.769    -0.502     1.640
 (1 - 6)     -0.637     0.642    -1.707     0.434
 (1 - 7)      0.123     1.000    -0.948     1.193
 (1 - 8)      0.307     0.993    -0.764     1.377
 (2 - 0)     -0.850     0.245    -1.921     0.220
 (2 - 1)     -0.526     0.837   